# Deep learning model and correction of bias

Hypothesis: removing variables ZIP, rent, job_stability and occupation reduces bias against minority in model. 

Removing variables rent, ZIP, occupation and job_stability removes bias from model according to demographic parity, equal opportunity and equalised odds measurements.

In [136]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from  sklearn.linear_model import LogisticRegression


In [137]:
seed = 123
np.random.seed(seed)

In [138]:
#import pre_processed train dataset
train = pd.read_csv('train_preprocessed.csv', index_col=0)
print('train shape: ' + str(train.shape))
train.head()

train shape: (471136, 13)


,minority,sex,ZIP,rent,education,age,income,loan_size,payment_timing,year,job_stability,occupation,default
0,1,0,1,1,57.230650,36.050927,205168.022244,7600.292199,3.302193,0,3.015554,1,1
1,1,0,1,1,45.891343,59.525251,187530.409981,5534.271289,3.843058,0,5.938132,1,1
2,1,0,1,1,46.775489,67.338108,196912.006690,2009.903438,2.059034,0,2.190777,1,1
3,1,0,1,1,41.784839,24.067401,132911.650615,3112.280893,3.936169,0,1.725860,1,1
4,1,0,1,1,41.744838,47.496605,161162.551205,1372.077093,3.709910,0,0.883104,1,1


In [139]:
#import pre_processed test dataset
test = pd.read_csv('test_preprocessed.csv', index_col=0)
print('test shape: ' + str(test.shape))
test.head()

test shape: (160000, 13)


,minority,sex,ZIP,rent,education,age,income,loan_size,payment_timing,job_stability,year,occupation,default
0,1,0,1,1,51.265723,25.710781,166455.209729,8064.951996,3.874735,43.764963,30,1,1
1,0,0,1,0,58.882849,39.689510,216752.885725,7166.701945,3.809001,46.903977,30,0,0
2,0,0,1,0,56.504545,25.847324,183764.480788,3322.045258,3.497214,63.453467,30,0,0
3,1,0,1,1,47.074111,26.381109,154057.004978,15.223904,3.535370,56.243840,30,1,0
4,1,0,1,1,48.916960,18.779902,143463.038107,7860.534547,3.663330,49.884194,30,1,0


In [140]:
#train dataset
X_tr_nob = train.drop(['default', 'minority', 'rent', 'job_stability', 'occupation', 'sex', 'ZIP'], axis=1)
X_tr_bias = train.drop(['default', 'minority', 'sex'], axis=1)
y_train= train['default']

#test dataset
X_te_nob = test.drop(['default', 'minority', 'rent', 'job_stability', 'occupation', 'sex', 'ZIP'], axis=1)
X_te_bias = test.drop(['default', 'minority', 'sex'], axis=1)
y_test = test['default']

In [141]:
from sklearn.preprocessing import StandardScaler

#Scale X train/test bias
scaler = StandardScaler()
X_fit_bias = scaler.fit(X_tr_bias)

X_train_bias = X_fit_bias.transform(X_tr_bias)
X_test_bias = X_fit_bias.transform(X_te_bias)

#Scale X train/test no bias
scaler = StandardScaler()
X_fit_nob = scaler.fit(X_tr_nob)

X_train_nob = X_fit_nob.transform(X_tr_nob)
X_test_nob = X_fit_nob.transform(X_te_nob)

# Linear classification: Base Model

In [163]:
#linear classification
from  sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#bias
lr_bias = LogisticRegression(solver='lbfgs')
lr_bias.fit(X_train_bias, y_train)

y_pred_lr_bias = lr_bias.predict(X_test_bias)
accuracy_lr_bias = accuracy_score(y_pred_lr_bias, y_test)


#no bias
lr_nob = LogisticRegression(solver='lbfgs')
lr_nob.fit(X_train_nob, y_train)

y_pred_lr_nob = lr_nob.predict(X_test_nob)
accuracy_lr_nob = accuracy_score(y_pred_lr_nob, y_test)

#print accuracy
print('Accuracy with bias: ' + str(round(accuracy_lr_bias*100,2)) + '%')
print('Accuracy with no bias: ' + str(round(accuracy_lr_nob*100,2)) + '%')

Accuracy with bias: 37.01%
Accuracy with no bias: 41.49%


In [164]:
#print classification report - BIAS
print(classification_report(y_test, y_pred_lr_bias))

              precision    recall  f1-score   support

           0       0.82      0.33      0.47    136055
           1       0.13      0.58      0.22     23945

    accuracy                           0.37    160000
   macro avg       0.48      0.46      0.34    160000
weighted avg       0.72      0.37      0.44    160000



In [166]:
#print classification report - NO BIAS
print(classification_report(y_test, y_pred_lr_nob))

              precision    recall  f1-score   support

           0       0.84      0.38      0.53    136055
           1       0.14      0.59      0.23     23945

    accuracy                           0.41    160000
   macro avg       0.49      0.49      0.38    160000
weighted avg       0.74      0.41      0.48    160000



In [167]:
confusion_matrix(y_test, y_pred_lr_nob)

array([[52267, 83788],
       [ 9835, 14110]])

# Deep Learning

In [145]:
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix

In [146]:
#stop if no improvement in loss after 3 epochs
es_callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

#class weight
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

/Users/Rolf/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=[0 1], y=0       1
1       1
2       1
3       1
4       1
       ..
7995    0
7996    0
7997    0
7998    0
7999    0
Name: default, Length: 471136, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [147]:
def create_model(df):
    ## Initialize model.
    model = Sequential() #if you want a recurrent NN, then specify here Recurrent()

    ## 1st Layer
    model.add(Dense(64, input_dim=df.shape[1]))
    model.add(Activation('relu'))
    model.add(Dropout(0.5)) #25% of neurons are de-activated randomly per batch. helps with generalisation.

    ## 2nd Layer
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.5)) #dropout number is arbitrary. trail and error. 

    ## Adding Softmax Layer
    model.add(Dense(1))
    model.add(Activation('sigmoid')) #softmax used for classification. softmax = [0,1]

    ## Define loss function
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], weighted_metrics=['accuracy'])

    return model

K-fold validation from this website:

https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

In [179]:
#Cross-validation
from sklearn.model_selection import StratifiedKFold

# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

def kfold_validation(X, y):

    X_df = pd.DataFrame(X)
    cvscores = []
    
    for train, test in kfold.split(X_df, y):
        model = create_model(X_df)
        model.fit(X_df.iloc[train], y.iloc[train], epochs=100, batch_size=1000, shuffle=True, verbose=0, 
                  callbacks=[es_callback], class_weight=class_weights)
        scores = model.evaluate(X_df.iloc[test], y.iloc[test], verbose=0)
        
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
        
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    
    return model

## Model with BIAS

In [180]:
FNN_bias = kfold_validation(X_train_bias, y_train)

accuracy: 99.98%
accuracy: 99.98%
accuracy: 99.98%
accuracy: 99.99%
accuracy: 99.99%
99.98% (+/- 0.00%)


In [198]:
#the kfold model is giving wierd results. 
#thus, switching to this model
#I think issue stems from stratifiedKfold not working, i.e. I'm doing something wrong
FNN_bias = create_model(X_train_bias)
FNN_bias.fit(X_train_bias, y_train, epochs=100, batch_size=1000, shuffle=True, verbose=2, callbacks=[es_callback],
           class_weight=class_weights)

Epoch 1/100
 - 2s - loss: 0.0638 - accuracy: 0.9785 - accuracy_1: 0.9785
Epoch 2/100
 - 2s - loss: 0.0037 - accuracy: 0.9996 - accuracy_1: 0.9996
Epoch 3/100
 - 1s - loss: 0.0029 - accuracy: 0.9997 - accuracy_1: 0.9997
Epoch 4/100
 - 2s - loss: 0.0025 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 5/100
 - 2s - loss: 0.0022 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 6/100
 - 2s - loss: 0.0022 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 7/100
 - 2s - loss: 0.0019 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 8/100
 - 2s - loss: 0.0019 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 9/100
 - 2s - loss: 0.0017 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 10/100
 - 2s - loss: 0.0019 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 11/100
 - 2s - loss: 0.0018 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 12/100
 - 2s - loss: 0.0017 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 13/100
 - 2s - loss: 0.0017 - accuracy: 0.9998 - accuracy_1: 0.9998
Epoch 14/100
 - 2s - loss: 0.0017 - accuracy: 0

In [201]:
y_pred_bias = FNN_bias.predict_classes(X_test_bias)

print(classification_report(y_test, y_pred_bias))

              precision    recall  f1-score   support

           0       0.85      0.47      0.60    136055
           1       0.15      0.52      0.23     23945

    accuracy                           0.48    160000
   macro avg       0.50      0.49      0.42    160000
weighted avg       0.74      0.48      0.55    160000



## Model with NO-BIAS

In [158]:
FNN_nob = kfold_validation(X_train_nob, y_train)

accuracy: 49.99%
accuracy: 50.01%
accuracy: 49.97%
accuracy: 49.85%
accuracy: 50.02%
49.97% (+/- 0.06%)


In [195]:
#the kfold model is giving wierd results. 
#thus, switching to this model
#I think issue stems from stratifiedKfold not working, i.e. I'm doing something wrong
FNN_nob = create_model(X_train_nob)
FNN_nob.fit(X_train_nob, y_train, epochs=100, batch_size=1000, shuffle=True, verbose=2, callbacks=[es_callback],
           class_weight=class_weights)

Epoch 1/100
 - 2s - loss: 0.6975 - accuracy: 0.5010 - accuracy_1: 0.5010
Epoch 2/100
 - 1s - loss: 0.6933 - accuracy: 0.5004 - accuracy_1: 0.5004
Epoch 3/100
 - 1s - loss: 0.6932 - accuracy: 0.5000 - accuracy_1: 0.5000
Epoch 4/100
 - 2s - loss: 0.6932 - accuracy: 0.5000 - accuracy_1: 0.5000
Epoch 5/100
 - 2s - loss: 0.6932 - accuracy: 0.4994 - accuracy_1: 0.4994
Epoch 6/100
 - 1s - loss: 0.6932 - accuracy: 0.5003 - accuracy_1: 0.5003
Epoch 7/100
 - 2s - loss: 0.6932 - accuracy: 0.4996 - accuracy_1: 0.4996
Epoch 8/100
 - 1s - loss: 0.6932 - accuracy: 0.5004 - accuracy_1: 0.5004
Epoch 9/100
 - 2s - loss: 0.6932 - accuracy: 0.4998 - accuracy_1: 0.4998
Epoch 10/100
 - 2s - loss: 0.6932 - accuracy: 0.5000 - accuracy_1: 0.5000
Epoch 11/100
 - 2s - loss: 0.6932 - accuracy: 0.5005 - accuracy_1: 0.5005
Epoch 12/100
 - 2s - loss: 0.6932 - accuracy: 0.5004 - accuracy_1: 0.5004


In [196]:
y_pred_nob = FNN_nob.predict_classes(X_test_nob)

print(classification_report(y_test, y_pred_nob))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90    136055
           1       0.15      0.04      0.07     23945

    accuracy                           0.82    160000
   macro avg       0.50      0.50      0.48    160000
weighted avg       0.75      0.82      0.78    160000



# Fairness of model

In [250]:
def bias_analysis(X_test_df, y_test_df, df_column_names, model, bias, minority):
    """ 
    X_test_df = X_test after StandardScaler transformation
    y_test_df = y_test 
    df_columns_names = X_test dataframe before StandardScaler transformation
    model = deep learning model
    bias = 0 or 1
    minority = 0 or 1
    """
    
    #join minority variable to X and y dataframes to differentiate between minority and non-minority
    X_test_min = np.concatenate((X_test_df, 
                                 test['minority'].values.reshape(-1,1)), 
                                axis=1)
    y_test_min = np.concatenate((y_test_df.values.reshape(-1,1),
                                 test['minority'].values.reshape(-1,1)), 
                                axis=1)
    
    #column headings for dataframe that will be converted from array
    columns = np.insert(df_column_names.columns, X_test_df.shape[1], 'minority')
    
    #convert numpy array to dataframe
    df_X_test = pd.DataFrame(X_test_min, columns=columns)
    df_y_test = pd.DataFrame(y_test_min, columns=['default','minority'])
    
    #subset minority==1, minority group
    X_minority = df_X_test[df_X_test['minority']==minority].values
    y_minority = df_y_test[df_y_test['minority']==minority].values
    
    #predict y
    y_pred_minority = model.predict_classes(X_minority[:,:X_test_df.shape[1]])

    #evaluate accuracy
    loss_dl_min, accuracy_dl_min, accuracy_weighted_dl_min = model.evaluate(X_minority[:,:X_test_df.shape[1]], y_minority[:,0])
    
    confusion = confusion_matrix(y_minority[:,0], y_pred_minority)
    
    if bias == 1:
        if minority == 1:
        
            print('Test loss bias minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy bias minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion

        else:

            print('Test loss bias non-minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy bias non-minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion
      
    #bias=0
    if bias == 0:
        if minority == 1:

            print('Test loss non-bias minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy non-bias minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion

        else:
            
            print('Test loss non-bias non-minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy non-bias non-minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion
        
    if bias==2:
                    
            print('Test loss downsampled non-minority: ' + str(round(loss_dl_min, 2)))
            print('Test accuracy downsampled non-minority: ' + str(round(accuracy_dl_min*100, 2)) + '%')

            print()

            return confusion

In [246]:
def bias_check(bias, minority):
    """
    bias = 0 or 1
        bias==1 means dataset is biased
    minority = 0 or 1
        minority==1 means minority is analysed
        
    bias ==2 
        when train dataset has been downsampled

    """
    
    if bias == 1:
        result = bias_analysis(X_test_bias, y_test, X_te_bias, FNN_bias, bias=bias, minority=minority)
        return result
    
    if bias == 0:
        result = bias_analysis(X_test_nob, y_test, X_te_nob, FNN_nob, bias=bias, minority=minority)
        return result
    
    if bias == 2:
        result = bias_analysis(X_test_bias, y_test, X_te_bias, FNN_ds, bias=bias, minority=minority)

### Calculate confusion martrix

In [293]:
#confusion matrix for BIAS and Minority
confusion_bias_minority = bias_check(bias=1, minority=1)

80348/80348 [==============================] - 2s 20us/step
Test loss bias minority: 19.9
Test accuracy bias minority: 16.81%



In [294]:
#confusion matrix for BIAS and Non-Minority
confusion_bias_non_minority = bias_check(bias=1, minority=0)

79652/79652 [==============================] - 1s 18us/step
Test loss bias non-minority: 3.46
Test accuracy bias non-minority: 78.87%



In [207]:
#confusion matrix for Non-BIAS and Minority
confusion_non_bias_minority = bias_check(bias=0, minority=1)

80348/80348 [==============================] - 2s 26us/step
Test loss non-bias minority: 0.69
Test accuracy non-bias minority: 81.84%



In [208]:
#confusion matrix for Non-Bias and Non-Minority
confusion_non_bias_non_minority = bias_check(bias=0, minority=0)

79652/79652 [==============================] - 2s 24us/step
Test loss non-bias non-minority: 0.69
Test accuracy non-bias non-minority: 81.86%



# Demographic Parity

Really good article explaining fairness measures in ML
https://towardsdatascience.com/how-to-define-fairness-to-detect-and-prevent-discriminatory-outcomes-in-machine-learning-ef23fd408ef2

Positive rate for both minority and non-minority should be the same. This means that the rate of (false positive + true positive) be similar for both groups. For example, the model should predict a 50% default rate in both minority and non-minority. 

Overall, demographic parity does not exist for this dataset since 99% default rate is predicted for the minority group and less than 1% for the non-minority group.

<img src=images/demographic_parity.png>

In [295]:
def demographic_parity(result, bias=0, minority=0):
    """
    bias = 0 or 1
        bias==1 means dataset is biased
    minority = 0 or 1
        minority==1 means minority is analysed

    """
    
    tn, fp, fn, tp = result.ravel()
    #tn, fp, fn, tp = bias_check(bias, minority).ravel()
    
    temp = (fp + tp) / (tn + fp + fn + tp)
    demo_parity = round(temp*100, 2) 
    
    if bias == 1:
        if minority == 0:
            print('Demographic parity for bias non-minority: ' + str(demo_parity) + '%')

        
        else: 
            print('Demographic parity for bias minority: ' + str(demo_parity) + '%')
   
    if bias == 0:
        if minority == 0:
            print('Demographic parity for non-bias non-minority: ' + str(demo_parity) + '%')    
            
        else: 
            print('Demographic parity for non-bias minority: ' + str(demo_parity) + '%')  
            
    if bias == 2:
        if minority == 0:
            print('Demographic parity for downsampled non-minority: ' + str(demo_parity) + '%')    
            
        else: 
            print('Demographic parity for downsampled minority: ' + str(demo_parity) + '%')  
            

In [273]:
#BIAS
demographic_parity(confusion_bias_minority, bias=1, minority=1)
demographic_parity(confusion_bias_non_minority, bias=1, minority=0)

Demographic parity for bias minority: 96.36%
Demographic parity for bias non-minority: 9.03%


In [274]:
#Non-BIAS
demographic_parity(confusion_non_bias_minority, bias=0, minority=1)
demographic_parity(confusion_non_bias_non_minority, bias=0, minority=0)

Demographic parity for non-bias minority: 4.48%
Demographic parity for non-bias non-minority: 4.55%


# Equal Opportunity

The false negative and true positive rate should be the same for both groups. 

Overall, equal opportunity exists since both groups correctly predict default rate to 100%. 

<img src=images/equal_opportunity.png>

In [301]:
def equal_opportunity(result, bias=0, minority=0):
    """
    bias = 0 or 1
        bias==1 means dataset is biased
    minority = 0 or 1
        minority==1 means minority is analysed

    """
    
    tn, fp, fn, tp = result.ravel()
    #tn, fp, fn, tp = bias_check(bias, minority).ravel()
    
    temp = tp / (fn + tp)
    equal_opportunity = round(temp*100, 2) 
    
    if bias == 1:
        if minority == 0:
            print('Equal opportunity for bias non-minority: ' + str(equal_opportunity) + '%')

        
        else: 
            print('Equal opportunity for bias minority: ' + str(equal_opportunity) + '%')

    
    if bias == 0:
        if minority == 0:
            print('Equal opportunity for non-bias non-minority: ' + str(equal_opportunity) + '%')

        
        else: 
            print('Equal opportunity for non-bias minority: ' + str(equal_opportunity) + '%')

        
    if bias==2:
        if minority == 0:
            print('Equal opportunity for downsampled non-minority: ' + str(equal_opportunity) + '%')

        
        else: 
            print('Equal opportunity for downsampled minority: ' + str(equal_opportunity) + '%')


In [232]:
#BIAS
equal_opportunity(confusion_bias_minority, bias=1, minority=1)
equal_opportunity(confusion_bias_non_minority, bias=1, minority=0)

Equal opportunity for bias minority: 93.82%
Equal opportunity for bias non-minority: 9.37%


In [227]:
#Non-BIAS
equal_opportunity(confusion_non_bias_minority, bias=0, minority=1)
equal_opportunity(confusion_non_bias_non_minority, bias=0, minority=0)

Equal opportunity for non-bias minority: 4.46%
Equal opportunity for non-bias non-minority: 4.42%


# Equalised Odds

<img src=images/equalised_odds.png>

In [300]:
def equalised_odds(result, bias=0, minority=0):
    """
    bias = 0 or 1
        bias==1 means dataset is biased
    minority = 0 or 1
        minority==1 means minority is analysed

    """
    
    tn, fp, fn, tp = result.ravel()
    #tn, fp, fn, tp = bias_check(bias, minority).ravel()
    
    temp = tp / (fn + tp)
    true_positive_rate = round(temp*100, 2)
    
    temp = tn / (fp + tn)
    true_negative_rate = round(temp*100, 2)
    
    if bias == 1:
        if minority == 0:
            print()
            print('True positive rate for bias non-minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for bias non-minority: ' + str(true_negative_rate) + '%')

        
        else: 
            print()
            print('True positive rate for bias minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for bias minority: ' + str(true_negative_rate) + '%')

    
    if bias == 0:
        if minority == 0:
            print()
            print('True positive rate for non-bias non-minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for non-bias non-minority: ' + str(true_negative_rate) + '%')

        
        else: 
            print()
            print('True positive rate for non-bias minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for non-bias minority: ' + str(true_negative_rate) + '%')

        
    if bias==2:
        if minority == 0:
            print()
            print('True positive rate for downsampled non-minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for downsampled non-minority: ' + str(true_negative_rate) + '%')

        
        else: 
            print()
            print('True positive rate for downsampled minority: ' + str(true_positive_rate) + '%')
            print('True negative rate for downsampled minority: ' + str(true_negative_rate) + '%')        

In [235]:
#BIAS
equalised_odds(confusion_bias_minority, bias=1, minority=1)
equalised_odds(confusion_bias_non_minority, bias=1, minority=0)

#Non-BIAS
equalised_odds(confusion_non_bias_minority, bias=0, minority=1)
equalised_odds(confusion_non_bias_non_minority, bias=0, minority=0)


True positive rate for bias minority: 93.82%
True negative rate for bias minority: 3.19%

True positive rate for bias non-minority: 9.37%
True negative rate for bias non-minority: 91.03%

True positive rate for non-bias minority: 4.46%
True negative rate for non-bias minority: 95.52%

True positive rate for non-bias non-minority: 4.42%
True negative rate for non-bias non-minority: 95.43%


# Downsampled Train Dataset

Inspired by this website for downsampling:
https://elitedatascience.com/imbalanced-classes

The smalled count among minority and default is 250 for minority==1 and default==0

In [238]:
#downsampling
from sklearn.utils import resample

# Separate majority and minority classes
df_non_minority_0 = train[(train['minority']==0) & (train['default']==0)]
df_non_minority_1 = train[(train['minority']==0) & (train['default']==1)]

df_minority_0 = train[(train['minority']==1) & (train['default']==0)]
df_minority_1 = train[(train['minority']==1) & (train['default']==1)]
 

# Downsample non_minority, default == 0
df_non_minority_0_downsampled = resample(df_non_minority_0, 
                                         replace=False,    # sample without replacement
                                         n_samples=250,     # to match minority class
                                         random_state=seed) # reproducible results

# Downsample non_minority, default == 1
df_non_minority_1_downsampled = resample(df_non_minority_1, 
                                         replace=False,    # sample without replacement
                                         n_samples=250,     # to match minority class
                                         random_state=seed) # reproducible results

# Downsample minority, default == 1
df_minority_1_downsampled = resample(df_minority_1, 
                                         replace=False,    # sample without replacement
                                         n_samples=250,     # to match minority class
                                         random_state=seed) # reproducible results

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_non_minority_0_downsampled,
                            df_non_minority_1_downsampled,
                            df_minority_0,
                            df_minority_1_downsampled
                           ])
 
# Display new class counts
print(df_downsampled['minority'].value_counts())

#Display distribution for minority, default
df_downsampled.groupby(["minority", "default"])["minority"].count()

1    500
0    500
Name: minority, dtype: int64


minority  default
0         0          250
          1          250
1         0          250
          1          250
Name: minority, dtype: int64

In [239]:
#create dataframe with independent variables (X) and indepedent variable (y)
X_ds = df_downsampled.drop(['default', 'minority', 'sex'], axis=1)
y_downsampled = df_downsampled['default']

#Scale X train/test bias
scaler = StandardScaler()
X_fit_bias_ds = scaler.fit(X_ds)

X_downsampled = X_fit_bias_ds.transform(X_ds)

## LogisticRegression - Downsampled

In [240]:
#Logistic Regression

lr_ds = LogisticRegression(solver='lbfgs')
lr_ds.fit(X_downsampled, y_downsampled)

y_pred_ds = lr_ds.predict(X_test_bias)
accuracy_lr_ds = accuracy_score(y_pred_ds, y_test)


#print accuracy
print('LogisticRegression accuracy with downsampled dataset: ' + str(round(accuracy_lr_ds*100,2)) + '%')
print()

#print classification report
print(classification_report(y_test, y_pred_ds))

LogisticRegression accuracy with downsampled dataset: 36.82%

              precision    recall  f1-score   support

           0       0.84      0.32      0.46    136055
           1       0.14      0.65      0.24     23945

    accuracy                           0.37    160000
   macro avg       0.49      0.48      0.35    160000
weighted avg       0.73      0.37      0.43    160000



## Deep learning - Downsampled

In [241]:
#train deep learning model with 5-kfold validation
FNN_ds = kfold_validation(X_downsampled, y_downsampled)

accuracy: 85.00%
accuracy: 90.00%
accuracy: 91.50%
accuracy: 91.50%
accuracy: 90.00%
89.60% (+/- 2.40%)


In [242]:
#the kfold model is giving wierd results. 
#thus, switching to this model
#I think issue stems from stratifiedKfold not working, i.e. I'm doing something wrong
FNN_bs = create_model(X_downsampled)
FNN_bs.fit(X_downsampled, y_downsampled, epochs=100, batch_size=1000, shuffle=True, verbose=2, callbacks=[es_callback],
           class_weight=class_weights)

Epoch 1/100
 - 0s - loss: 0.7952 - accuracy: 0.4740 - accuracy_1: 0.4740
Epoch 2/100
 - 0s - loss: 0.7789 - accuracy: 0.4740 - accuracy_1: 0.4740
Epoch 3/100
 - 0s - loss: 0.7671 - accuracy: 0.5080 - accuracy_1: 0.5080
Epoch 4/100
 - 0s - loss: 0.7422 - accuracy: 0.5050 - accuracy_1: 0.5050
Epoch 5/100
 - 0s - loss: 0.7490 - accuracy: 0.5020 - accuracy_1: 0.5020
Epoch 6/100
 - 0s - loss: 0.7203 - accuracy: 0.5360 - accuracy_1: 0.5360
Epoch 7/100
 - 0s - loss: 0.7139 - accuracy: 0.5440 - accuracy_1: 0.5440
Epoch 8/100
 - 0s - loss: 0.6906 - accuracy: 0.5820 - accuracy_1: 0.5820
Epoch 9/100
 - 0s - loss: 0.6796 - accuracy: 0.5810 - accuracy_1: 0.5810
Epoch 10/100
 - 0s - loss: 0.6614 - accuracy: 0.5920 - accuracy_1: 0.5920
Epoch 11/100
 - 0s - loss: 0.6668 - accuracy: 0.5910 - accuracy_1: 0.5910
Epoch 12/100
 - 0s - loss: 0.6473 - accuracy: 0.6170 - accuracy_1: 0.6170
Epoch 13/100
 - 0s - loss: 0.6319 - accuracy: 0.6270 - accuracy_1: 0.6270
Epoch 14/100
 - 0s - loss: 0.6154 - accuracy: 0

In [292]:
#accuracy of downsampled dataset on test dataset
y_pred_dl_ds = FNN_ds.predict_classes(X_test_bias, verbose=0)

accuracy_dl_ds = accuracy_score(y_pred_dl_ds, y_test)

#print accuracy
print('Deep Learning accuracy with downsampled dataset: ' + str(round(accuracy_dl_ds*100,2)) + '%')
print()

print(classification_report(y_test, y_pred_dl_ds))

Deep Learning accuracy with downsampled dataset: 52.44%

              precision    recall  f1-score   support

           0       0.91      0.49      0.64    136055
           1       0.20      0.71      0.31     23945

    accuracy                           0.52    160000
   macro avg       0.55      0.60      0.47    160000
weighted avg       0.80      0.52      0.59    160000



## Fairness - Identifying Bias

In [281]:
confusion_ds_minority = bias_analysis(X_test_bias, y_test, X_te_bias, FNN_ds, bias=2, minority=1)

80348/80348 [==============================] - 1s 19us/step
Test loss downsampled non-minority: 0.82
Test accuracy downsampled non-minority: 50.62%



In [282]:
confusion_ds_non_minority = bias_analysis(X_test_bias, y_test, X_te_bias, FNN_ds, bias=2, minority=0)

79652/79652 [==============================] - 2s 19us/step
Test loss downsampled non-minority: 0.78
Test accuracy downsampled non-minority: 54.28%



### Demographic Parity - Downsampled

In [296]:
demographic_parity(confusion_ds_minority, bias=2, minority=1)

Demographic parity for downsampled minority: 56.53%


In [297]:
demographic_parity(confusion_ds_non_minority, bias=2, minority=1)

Demographic parity for downsampled minority: 50.98%


### Equal Opportunity - Downsampled

In [304]:
#equal opportunity for downsampled minority. 
equal_opportunity(confusion_ds_minority, bias=2, minority=1)

Equal opportunity for downsampled minority: 73.77%


In [305]:
#equal opportunity for downsampled non-minority. 
equal_opportunity(confusion_ds_non_minority, bias=2, minority=0)

Equal opportunity for downsampled non-minority: 67.66%


### Equalised Odds - Downsampled

In [302]:
#equalised odds for downsampled minority
equalised_odds(confusion_ds_minority, bias=2, minority=1)


True positive rate for downsampled minority: 73.77%
True negative rate for downsampled minority: 46.52%


In [303]:
#equalised odds for donwsampled non-minority
equalised_odds(confusion_ds_non_minority, bias=2, minority=0)


True positive rate for downsampled non-minority: 67.66%
True negative rate for downsampled non-minority: 51.94%


# Disparate Impact Remover

Notebook on how to remove bias in a variable: https://nbviewer.jupyter.org/github/srnghn/bias-mitigation-examples/blob/master/Bias%20Mitigation%20with%20Disparate%20Impact%20Remover.ipynb

In [309]:
#%pip install aif360

In [310]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from numba.core.decorators import jit

In [311]:
train_BLD = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=train,
                                label_names=['default'],
                                protected_attribute_names=['minority'],
                                unprivileged_protected_attributes=['1'])
test_BLD = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=test,
                                label_names=['default'],
                                protected_attribute_names=['minority'],
                                unprivileged_protected_attributes=['1'])

In [312]:
#transform dataset to remove disparate impact
di = DisparateImpactRemover(repair_level=1.0)
rp_train = di.fit_transform(train_BLD)
rp_test = di.fit_transform(test_BLD)

In [313]:
#train deep learning model
FNN_rp = create_model(rp_train.features[:, 2:]) #first two columns dropped since sex and minority
FNN_rp.fit(rp_train.features[:, 2:], rp_train.labels, epochs=100, batch_size=1000, shuffle=True, verbose=2, callbacks=[es_callback])

Epoch 1/100
 - 2s - loss: 786.4068 - accuracy: 0.4994 - accuracy_1: 0.4994
Epoch 2/100
 - 1s - loss: 4.2479 - accuracy: 0.4994 - accuracy_1: 0.4994
Epoch 3/100
 - 1s - loss: 1.5802 - accuracy: 0.5002 - accuracy_1: 0.5002
Epoch 4/100
 - 1s - loss: 1.3353 - accuracy: 0.5001 - accuracy_1: 0.5001
Epoch 5/100
 - 1s - loss: 1.1594 - accuracy: 0.5004 - accuracy_1: 0.5004
Epoch 6/100
 - 1s - loss: 1.0717 - accuracy: 0.4995 - accuracy_1: 0.4995
Epoch 7/100
 - 1s - loss: 0.9451 - accuracy: 0.4989 - accuracy_1: 0.4989
Epoch 8/100
 - 1s - loss: 0.8724 - accuracy: 0.4984 - accuracy_1: 0.4984
Epoch 9/100
 - 1s - loss: 0.8223 - accuracy: 0.5007 - accuracy_1: 0.5007
Epoch 10/100
 - 1s - loss: 0.7902 - accuracy: 0.4993 - accuracy_1: 0.4993
Epoch 11/100
 - 1s - loss: 0.7520 - accuracy: 0.4997 - accuracy_1: 0.4997
Epoch 12/100
 - 1s - loss: 0.7551 - accuracy: 0.5004 - accuracy_1: 0.5004
Epoch 13/100
 - 1s - loss: 0.7143 - accuracy: 0.5000 - accuracy_1: 0.5000
Epoch 14/100
 - 1s - loss: 0.7155 - accuracy:

In [314]:
di_preds = FNN_rp.predict(rp_test.features[:, 2:])

In [315]:
di_pred_np = np.concatenate((test[['minority']],
                                di_preds), 
                                axis=1)
di_pred_df = pd.DataFrame(di_pred_np, columns=['minority', 'preds'])
di_pred_df.head()

,minority,preds
0,1.0,0.502461
1,0.0,0.502461
2,0.0,0.502461
3,1.0,0.502461
4,1.0,0.502461


In [316]:
di_pred_df['preds'].mean()

0.5024610757827759

In [328]:
#use model not trained disparate impact remover
#predict using test transformed by disparate impact remover
y_pred_dl_bias_di = FNN_rp.predict_classes(rp_test.features[:, 2:], verbose=0)

accuracy_dl_bias_di = accuracy_score(y_pred_dl_bias_di, y_test)

#print accuracy
print('Deep Learning accuracy after disparate impact remover on bias dataset: ' + 
      str(round(accuracy_dl_bias_di*100,2)) + '%')
print()

print(classification_report(y_test, y_pred_dl_bias_di))

print()

print(confusion_matrix(y_test, y_pred_dl_bias_di))

Deep Learning accuracy after disparate impact remover on bias dataset: 14.97%

              precision    recall  f1-score   support

           0       0.00      0.00      0.00    136055
           1       0.15      1.00      0.26     23945

    accuracy                           0.15    160000
   macro avg       0.07      0.50      0.13    160000
weighted avg       0.02      0.15      0.04    160000


[[     0 136055]
 [     0  23945]]


## Disparate Impact Remover - Downsampled

In [318]:
#create disparate impact remover
#preparation for next code cell below
train_BLD_ds = BinaryLabelDataset(favorable_label='1',
                                unfavorable_label='0',
                                df=df_downsampled,
                                label_names=['default'],
                                protected_attribute_names=['minority'],
                                unprivileged_protected_attributes=['1'])

In [319]:
#fit disaparate impact remover on train
di = DisparateImpactRemover(repair_level=1.0)
rp_train_ds = di.fit_transform(train_BLD_ds)

In [320]:
#downsampled dataset with disparate impact remover
FNN_rp_ds = create_model(rp_train.features[:, 2:]) #first two columns dropped since sex and minority
FNN_rp_ds.fit(rp_train_ds.features[:, 2:], rp_train_ds.labels, epochs=100, batch_size=1000, 
           shuffle=True, verbose=2, callbacks=[es_callback])

Epoch 1/100
 - 0s - loss: 10170.0479 - accuracy: 0.5060 - accuracy_1: 0.5060
Epoch 2/100
 - 0s - loss: 10011.2129 - accuracy: 0.4900 - accuracy_1: 0.4900
Epoch 3/100
 - 0s - loss: 8978.9189 - accuracy: 0.4880 - accuracy_1: 0.4880
Epoch 4/100
 - 0s - loss: 9107.9141 - accuracy: 0.4860 - accuracy_1: 0.4860
Epoch 5/100
 - 0s - loss: 8244.8262 - accuracy: 0.5110 - accuracy_1: 0.5110
Epoch 6/100
 - 0s - loss: 7607.5508 - accuracy: 0.4960 - accuracy_1: 0.4960
Epoch 7/100
 - 0s - loss: 6282.1855 - accuracy: 0.5190 - accuracy_1: 0.5190
Epoch 8/100
 - 0s - loss: 6612.3560 - accuracy: 0.4940 - accuracy_1: 0.4940
Epoch 9/100
 - 0s - loss: 6744.8779 - accuracy: 0.4820 - accuracy_1: 0.4820
Epoch 10/100
 - 0s - loss: 5892.8799 - accuracy: 0.4980 - accuracy_1: 0.4980
Epoch 11/100
 - 0s - loss: 4860.2256 - accuracy: 0.4980 - accuracy_1: 0.4980
Epoch 12/100
 - 0s - loss: 4971.5049 - accuracy: 0.4880 - accuracy_1: 0.4880
Epoch 13/100
 - 0s - loss: 5020.5508 - accuracy: 0.5090 - accuracy_1: 0.5090
Epoch 

In [321]:
#predict default rate
di_preds_ds = FNN_rp_ds.predict(rp_test.features[:, 2:])

#concat predictions with minority
di_pred_np_ds = np.concatenate((test[['minority']],
                                di_preds_ds), 
                                axis=1)

#convert to dataframe
di_pred_df_ds = pd.DataFrame(di_pred_np_ds, columns=['minority', 'preds'])
di_pred_df_ds.head()

,minority,preds
0,1.0,0.0
1,0.0,0.0
2,0.0,0.0
3,1.0,0.0
4,1.0,0.0


In [322]:
#only 1s are predicted
di_pred_df_ds['preds'].mean()

0.0

In [327]:
#use model not trained disparate impact remover
#predict using test transformed by disparate impact remover
y_pred_dl_ds_di = FNN_rp_ds.predict_classes(rp_test.features[:, 2:], verbose=0)

accuracy_dl_ds_di = accuracy_score(y_pred_dl_ds_di, y_test)

#print accuracy
print('Deep Learning accuracy after disparate impact remover on downsampled dataset: ' + 
      str(round(accuracy_dl_ds_di*100,2)) + '%')
print()

print(classification_report(y_test, y_pred_dl_ds_di))

print()
print(confusion_matrix(y_test, y_pred_dl_ds_di))

Deep Learning accuracy after disparate impact remover on downsampled dataset: 85.03%

              precision    recall  f1-score   support

           0       0.85      1.00      0.92    136055
           1       0.00      0.00      0.00     23945

    accuracy                           0.85    160000
   macro avg       0.43      0.50      0.46    160000
weighted avg       0.72      0.85      0.78    160000


[[136055      0]
 [ 23945      0]]
